# Hidden Markov Models for English POS Tagging

In this notebook, you will implement a first-order Hidden Markov Models for English POS Tagging. We train an HMM model on a training data and evaluate the accuracy of the model on a separate test data.

## Dataset

We will use the Treebak data obtained from nltk.


In [ ]:
import nltk
from nltk.corpus import treebank

nltk.download('universal_tagset')
nltk.download('treebank')

[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

Load data set

In [ ]:
tagged_sentences = treebank.tagged_sents(tagset='universal')
len(tagged_sentences)

3914

In [ ]:
tagged_sentences[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

### Tagset

Let's figure out how many unique tags in the training data.

In [ ]:
tagset = set()
for s in tagged_sentences:
    for _,tag in s:
        tagset.add(tag)
print('# Number of tags: %d' % len(tagset))
print('Tagset = %s' % tagset)

# Number of tags: 12
Tagset = {'NUM', 'PRON', '.', 'PRT', 'ADP', 'X', 'ADV', 'NOUN', 'CONJ', 'VERB', 'ADJ', 'DET'}


### Create train/test/split

In [ ]:
from sklearn.model_selection import train_test_split

train_tagged_sentences, test_tagged_sentences = train_test_split(tagged_sentences, test_size=0.2, random_state=42)

We make test untagged sentences and corresponding tags

In [ ]:
test_sentences = []
test_tag_sequences = []

for sen in test_tagged_sentences:
    words, tags = zip(*sen)
    test_sentences.append(words)
    test_tag_sequences.append(tags)

print(test_sentences[0])
print(test_tag_sequences[0])

('For', 'the', 'Agency', 'for', 'International', 'Development', ',', 'appropriators', 'approved', '$', '200', 'million', '*U*', 'in', 'secondary', 'loan', 'guarantees', 'under', 'an', 'expanded', 'trade', 'credit', 'insurance', 'program', ',', 'and', 'total', 'loan', 'guarantees', 'for', 'the', 'Overseas', 'Private', 'Investment', 'Corp.', 'are', 'increased', '*-3', 'by', '$', '40', 'million', '*U*', 'over', 'fiscal', '1989', 'as', 'part', 'of', 'the', 'same', 'Poland', 'package', '.')
('ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', '.', 'NOUN', 'VERB', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'VERB', 'NOUN', 'NOUN', 'NOUN', 'NOUN', '.', 'CONJ', 'ADJ', 'NOUN', 'NOUN', 'ADP', 'DET', 'NOUN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'VERB', 'X', 'ADP', '.', 'NUM', 'NUM', 'X', 'ADP', 'ADJ', 'NUM', 'ADP', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', '.')


## Training First-Order Hidden Markov Models for POS Tagging

Please refer to the pseudo code in the lecture slide. We implement the function `train` model that iterate tagged sentences and calculate parameters of HMM model ans save parameters to a model file.

Parameters of our HMM model include transition probabilities and emission probabilities. Probabilities are calculated as follows.

Transition probabilities from POS $\rightarrow$ POS.

$$
P_T(t_i|t_{i-1})=\frac{C(t_{i-1}t_i)}{C(t_{i-1})}
$$

where $C(t_{i-1})$ is number of occurences of the tag $t_{i-1}$ in the data; and $C(t_{i-1}t_i)$ is the number of times that the tag $t_{i-1}$ followed by the tag $t_i$ in our training data.

Emission probabilities are calculated as follows.

$$
P_E(w_i|t_i)=\frac{C(t_i,w_i)}{C(t_i)}
$$

In [ ]:
from collections import defaultdict


def train(train_sentences, model_file: str):
    emit = defaultdict(int)   # dictionary to store emission count C(t_i, w_i)
    transition = defaultdict(int)   # transition count C(t_{i-1}, t_i)
    context = defaultdict(int)  # count the context
    for sen in train_sentences:
        previous = '<s>'    # Make the sentence start
        context[previous] += 1
        
        for word, tag in sen:
            transition[previous + ' ' + tag] += 1   # Count the transition
            context[tag] += 1   # Count the context
            emit[tag + ' ' + word] += 1   # Count the emission
            previous = tag

        transition[previous + ' </s>'] += 1   # Sentence stop

    # Now we will save the parameters of the model to a file
    with open(model_file, 'w') as fo:

        # Save transition probabilities
        for key, value in transition.items():
            previous, word = key.split(' ')
            fo.write('T %s %f\n' % (key, value/context[previous]))

        # Save emission probabilities
        for key, value in emit.items():
            tag, word = key.split(' ')
            fo.write('E %s %f\n' % (key, value/context[tag]))

    print('Finished training first-order HMM!')

In the `train` function, we will use dict `emit` to store count values $C(t_i, w_i)$ and `transition` to store count values $C(t_{i-1}t_i)$.  **The keys of dictionaries are built by joining two strings (two POS tags or a pair of (tag, word) with a single space**.

In the model file, we save each probability in a line. In the beginning of each line, we write "T" to indicate transition probabilities and "E" for emission probabilities.

The content of the model file is something like as follows.

```
T <s> NOUN 0.292558
T NOUN NOUN 0.262607
T NOUN . 0.241755
T . NUM 0.076488
T NUM NOUN 0.358993

...
E NOUN monster 0.000044
E NUM the 0.000355
E VERB implant 0.000092
E NOUN brain 0.000044
E NOUN patient 0.000044
```

In [ ]:
train(train_tagged_sentences, 'HMM_model.txt')

Finished training first-order HMM!


Let's see some first lines of the model.

In [ ]:
!head -n 5 HMM_model.txt

T <s> NOUN 0.292558
T NOUN NOUN 0.262607
T NOUN . 0.241755
T . NUM 0.076488
T NUM NOUN 0.358993


In [ ]:
!tail -n 5 HMM_model.txt

E NOUN monster 0.000044
E NUM the 0.000355
E VERB implant 0.000092
E NOUN brain 0.000044
E NOUN patient 0.000044


## Loading HMM model from file

We are going to write code to load the model file which we trained above into three dictionaries:

- `transition` is the dictionary to store transition probabilities $P_T(t_i|t_{i-1})$.
- `emission` is the dictionary to store emission probabilities $P_E(w_i|t_i)$.
- `possible_tags` is the dictionary to store unique tags in the model. It map from a POS tag to 1.

In [ ]:
def load_model(model_file: str):
    """Load saved HMM model
    """
    transition = defaultdict(lambda: 0)
    emission = defaultdict(lambda: 0)
    possible_tags = {}
    
    with open(model_file, 'r') as f:
        for line in f:
            line = line.strip()
            _type, context, word, prob = line.split(' ')
            prob = float(prob)
            possible_tags[context] = 1  # # We use this to enumerate all tags
            if _type == 'T':
                transition[context + ' ' + word] = prob
            else:
                emission[context + ' ' + word] = prob
    return transition, emission, possible_tags

Let's try to call the function and see the content of results.

In [ ]:
transition, emission, possible_tags = load_model('HMM_model.txt')
print( list(possible_tags.keys()) )
print(transition)
print(emission)

['<s>', 'NOUN', '.', 'NUM', 'ADJ', 'VERB', 'DET', 'ADP', 'CONJ', 'PRON', 'X', 'ADV', 'PRT']
defaultdict(<function load_model.<locals>.<lambda> at 0x7f9590f26040>, {'<s> NOUN': 0.292558, 'NOUN NOUN': 0.262607, 'NOUN .': 0.241755, '. NUM': 0.076488, 'NUM NOUN': 0.358993, 'NOUN ADJ': 0.011691, 'ADJ .': 0.065341, '. VERB': 0.087384, 'VERB VERB': 0.171009, 'VERB DET': 0.132391, 'DET NOUN': 0.637229, 'NOUN ADP': 0.175886, 'ADP DET': 0.327628, 'DET ADJ': 0.207359, 'ADJ NOUN': 0.699469, 'NOUN NUM': 0.009728, 'NUM .': 0.119191, '. </s>': 0.332016, '<s> CONJ': 0.051421, 'CONJ PRON': 0.061315, 'PRON VERB': 0.481481, 'VERB ADJ': 0.064763, '<s> DET': 0.22932, 'NOUN VERB': 0.147313, 'VERB X': 0.219235, 'X DET': 0.054064, 'ADJ X': 0.021059, 'X ADV': 0.026842, 'ADV ADP': 0.122586, 'ADP NOUN': 0.323413, 'VERB PRON': 0.03677, 'PRON NOUN': 0.21093, 'NOUN CONJ': 0.042226, 'CONJ VERB': 0.151059, 'VERB ADP': 0.091371, 'ADP NUM': 0.063099, '. DET': 0.095503, 'DET X': 0.045022, 'X VERB': 0.202741, 'DET VERB':

## Viterbi algorithm

We will implement the viterbi algorithm to determine the tag sequence for a tokenized sentence given the probabilities we have loaded from the model file.

In [ ]:
import math


def viterbi(words, transition, emission, possible_tags):
    """Infer the tag sequence for a tokenized sentence

    Args:
        words (list): a list of tokens
        transition (dict): transition probabilities
        emission (dict): emission probabilities
    """
    l = len(words)
    best_score = {}
    best_edge = {}
    best_score[('0 <s>')] = 0  # Start with <s>
    best_edge[('0 <s>')] = None

    K = frozenset(possible_tags.keys())
    K0 = frozenset(['<s>'])
    # Forward Step
    for i in range(l):
        tagset_prev, tagset_next = K, K
        if i == 0:
            tagset_prev = K0
        for prev in tagset_prev:
            for _next in tagset_next:
                if str(i) + ' ' + prev in best_score and prev + ' ' + _next in transition:
                    if emission[_next + ' ' + words[i]] == 0:
                        # To avoid zero probabilities, we use very small value
                        emission[_next + " " + words[i]] = 10 ** (-10)
                    
                    score = best_score[str(i) + ' ' + prev] + (-math.log(transition[prev + ' ' + _next])) + (-math.log(emission[_next + ' ' + words[i]]))

                    if str(i + 1) + " " + _next not in best_score or best_score[str(i + 1) + " " + _next] > score:
                        best_score[str(i + 1) + " " + _next] = score
                        best_edge[str(i + 1) + " " + _next] = str(i) + " " + prev
    
    tagset_prev = K
    if l == 0:
        tagset_prev = K0
    for prev in tagset_prev:
        if str(l) + ' ' + prev in best_score:
            if (prev + ' ' + '</s>') not in transition:
                transition[prev + ' ' + '</s>'] = 10 ** (-10)
            
            # Calculate best_score[str(l+1) + ' </s>'] and best_edge[str(l+1) + ' </s>'] 
            # for the sentence top symbole '</s>'
            # The different from the other time step is that, we do not use emission probility in calculating score
            score = best_score[str(l) + ' ' + prev] + (-math.log(transition[prev + ' ' + '</s>']))
            if str(l+1) + ' ' + '</s>' not in best_score or best_score[str(l+1) + ' </s>'] > score:
                best_score[str(l+1) + ' </s>'] = score
                best_edge[str(l+1) + ' </s>'] = str(l) + ' ' + prev
    
    # Backward Step
    tags = []
    next_edge = best_edge[str(l + 1) + " " + "</s>"]
    # TODO: Complete the backward step in Viterbi algorithm
    # Finish the while loop in the pseudo code
    while next_edge != "0 <s>":
        position, tag= next_edge.split()
        tags.append(tag)
        next_edge = best_edge[next_edge]
    
    # END OF YOUR CODE
    tags.reverse()
    return tags

Let's try to use the function viterbi function to find the tag sequence for a given sentence.

In [ ]:
# test sentence
print(test_sentences[1], test_tag_sequences[1])

('The', 'market', 'is', 'just', 'becoming', 'more', 'efficient', '.', "''") ('DET', 'NOUN', 'VERB', 'ADV', 'VERB', 'ADV', 'ADJ', '.', '.')


In [ ]:
viterbi(test_sentences[1], transition, emission, possible_tags)

['DET', 'NOUN', 'VERB', 'ADV', 'VERB', 'ADV', 'ADJ', '.', '.']

## Model Evaluation

We will evaluate the model on the test data with accuracy measure.

In [ ]:
def evaluate(test_tagged_sentences, transition, emission, possible_tags):
    correct = 0
    total = 0

    for sen in test_tagged_sentences:
        words, gold_tags = zip(*sen)
        predicted_tags = viterbi(words, transition, emission, possible_tags)
        for t1, t2 in zip(predicted_tags, gold_tags):
            total += 1
            if t1 == t2:
                correct += 1
    return 100.0 * correct/total, correct, total

In [ ]:
acc, correct, total = evaluate(test_tagged_sentences, transition, emission, possible_tags)
print("Accuracy = {} ({}/{})".format(acc, correct, total))

Accuracy = 93.644459584408 (19243/20549)


## HMM Tagger in NLTK


In [ ]:
import nltk

tagger = nltk.HiddenMarkovModelTagger.train(train_tagged_sentences)

In [ ]:
tagger.accuracy(test_tagged_sentences)

0.9273930604895615

Predict the tag sequence for an input

In [ ]:
sen = ['The', 'market', 'is', 'just', 'becoming', 'more', 'efficient', '.', "''"]
tagger.tag(sen)

[('The', 'DET'),
 ('market', 'NOUN'),
 ('is', 'VERB'),
 ('just', 'ADV'),
 ('becoming', 'VERB'),
 ('more', 'ADV'),
 ('efficient', 'ADJ'),
 ('.', '.'),
 ("''", '.')]

In [ ]:
res = tagger.tag(sen)
_, tags = zip(*res)
tags = list(tags)
print(tags)

['DET', 'NOUN', 'VERB', 'ADV', 'VERB', 'ADV', 'ADJ', '.', '.']
